# Multi-Head Attention: Collaborate Instead of Concatenate

### Abstract
Attention layers are widely used in natural language processing (NLP) and are
beginning to influence computer vision architectures. However, they suffer from
over-parameterization. For instance, it was shown that the majority of attention
heads could be pruned without impacting accuracy. This work aims to enhance
current understanding on how multiple heads interact. Motivated by the observation
that trained attention heads share common key/query projections, we propose a
collaborative multi-head attention layer that enables heads to learn shared projections. Our scheme improves the computational cost and number of parameters in
an attention layer and can be used as a drop-in replacement in any transformer
architecture. For instance, by allowing heads to collaborate on a neural machine
translation task, we can reduce the key dimension by a factor of eight without any
loss in performance. We also show that it is possible to re-parametrize a pre-trained
multi-head attention layer into our collaborative attention layer. Even without
retraining, collaborative multi-head attention manages to reduce the size of the key
and query projections by half without sacrificing accuracy. Our code is public


### Trừu tượng
Các lớp chú ý được sử dụng rộng rãi trong xử lý ngôn ngữ tự nhiên (NLP) và bắt đầu ảnh hưởng đến kiến ​​trúc thị giác máy tính. Tuy nhiên, họ bị tham số hóa quá mức. Ví dụ, nó đã chỉ ra rằng phần lớn các đầu chú ý có thể được cắt tỉa mà không ảnh hưởng đến độ chính xác. Công việc này nhằm tăng cường sự hiểu biết hiện tại về cách nhiều người đứng đầu tương tác. Được thúc đẩy bởi sự quan sát rằng các đầu chú ý được đào tạo chia sẻ các phép chiếu khóa / truy vấn chung, chúng tôi đề xuất một lớp chú ý nhiều đầu hợp tác cho phép các đầu tìm hiểu các phép chiếu được chia sẻ. Đề án của chúng tôi cải thiện chi phí tính toán và số lượng tham số trong một lớp chú ý và có thể được sử dụng như một sự thay thế thả trong bất kỳ kiến ​​trúc máy biến áp nào. Chẳng hạn, bằng cách cho phép người đứng đầu cộng tác trong một nhiệm vụ dịch máy thần kinh, chúng ta có thể giảm kích thước khóa xuống 8 lần mà không làm giảm hiệu suất. Chúng tôi cũng chỉ ra rằng có thể tái tham gia một lớp chú ý đa đầu được đào tạo trước vào lớp chú ý hợp tác của chúng tôi. Ngay cả khi không đào tạo lại, sự chú ý của nhiều người hợp tác quản lý để giảm một nửa kích thước của khóa và các phép chiếu truy vấn mà không làm giảm độ chính xác. Mã của chúng tôi là công khai

# 1 Introduction
Since the invention of attention (Bahdanau et al., 2014) and its popularization in the transformer
architecture (Vaswani et al., 2017), multi-head attention (MHA) has become the de facto architecture
for natural language understanding tasks (Devlin et al., 2019) and neural machine translation. Attention mechanisms have also gained traction in computer vision following the work of Ramachandran
et al. (2019) and Bello et al. (2019). Nevertheless, despite their wide adoption, we currently lack
solid theoretical understanding of how transformers operate. In fact, many of their modules and
hyperparameters are derived from empirical evidences that are possibly circumstantial.

The uncertainty is amplified in multi-head attention, where both the roles and interactions between
heads are still poorly understood. Empirically, it is well known that using multiple heads can improve
model accuracy. However, not all heads are equally informative, and it has been shown that certain
heads can be pruned without impacting model performance. For instance, Voita et al. (2019) present
a method to quantify head utility and prune redundant members. Michel et al. (2019) go further to
question the utility of multiple heads by testing the effect of heavy pruning in several settings. On
the other hand, Cordonnier et al. (2020) prove that multiple heads are needed for self-attention to
perform convolution, specifically requiring one head per pixel in the filter’s receptive field.

This work aims to better detect and quantify head redundancy by asking whether independent heads
learn overlapping or distinct concepts. We discover that some key/query projected dimensions are
redundant, as trained concatenated heads tend to compute their attention patterns on common features.
Our finding implies that MHA can be re-parametrized with better weight sharing for these common
projections and a lower number of parameters

# 1. Giới thiệu
Kể từ khi phát minh ra sự chú ý (Bahdanau et al., 2014) và sự phổ biến của nó trong kiến ​​trúc máy biến áp (Vaswani et al., 2017), sự chú ý nhiều đầu (MHA) đã trở thành kiến ​​trúc thực tế cho các nhiệm vụ hiểu ngôn ngữ tự nhiên (Devlin et al., 2019) và dịch máy thần kinh. Các cơ chế chú ý cũng đã đạt được lực kéo trong tầm nhìn máy tính sau công việc của Ramachandran et al. (2019) và Bello và cộng sự. (2019). Tuy nhiên, mặc dù áp dụng rộng rãi, chúng tôi hiện thiếu hiểu biết lý thuyết vững chắc về cách thức hoạt động của máy biến áp. Trong thực tế, nhiều mô-đun và siêu âm của chúng có nguồn gốc từ các bằng chứng thực nghiệm có thể là hoàn cảnh.

Sự không chắc chắn được khuếch đại trong sự chú ý của nhiều đầu, trong đó cả vai trò và tương tác giữa các đầu vẫn chưa được hiểu rõ. Theo kinh nghiệm, người ta biết rằng sử dụng nhiều đầu có thể cải thiện độ chính xác của mô hình. Tuy nhiên, không phải tất cả các đầu đều có thông tin như nhau, và nó đã được chứng minh rằng một số đầu có thể được cắt tỉa mà không ảnh hưởng đến hiệu suất mô hình. Ví dụ, Voita et al. (2019) trình bày một phương pháp để định lượng tiện ích đầu và cắt tỉa các thành viên dư thừa. Michel và cộng sự. (2019) đi xa hơn để đặt câu hỏi về tiện ích của nhiều đầu bằng cách kiểm tra hiệu quả của việc cắt tỉa nặng trong một số cài đặt. Mặt khác, Cordonnier et al. (2020) chứng minh rằng cần có nhiều đầu để tự chú ý thực hiện tích chập, đặc biệt yêu cầu một đầu trên mỗi pixel trong trường tiếp nhận bộ lọc.

Công việc này nhằm mục đích phát hiện và định lượng tốt hơn sự dư thừa của đầu bằng cách hỏi xem những người đứng đầu độc lập có học các khái niệm chồng chéo hay khác biệt hay không. Chúng tôi phát hiện ra rằng một số kích thước dự kiến ​​khóa / truy vấn là dư thừa, vì các đầu nối được đào tạo có xu hướng tính toán các mẫu chú ý của chúng trên các tính năng phổ biến. Phát hiện của chúng tôi ngụ ý rằng MHA có thể được tái tham số hóa với việc chia sẻ trọng lượng tốt hơn cho các dự đoán chung này và số lượng tham số thấp hơn